In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import shap
import lime
from lime.lime_tabular import LimeTabularExplainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import mean_squared_error
shap.initjs()
import json
import re
import nltk
from wordcloud import WordCloud
from nltk.corpus import stopwords
from collections import Counter
from PyPDF2 import PdfReader
import re 
import csv


# Con questo codice si trasferisce il da file pdf a csv il file di domande di nostro interesse per gli anni 2016, 2017, 2018, 2019



## Area preclinica 2016

In [ ]:
# Per prima cosa si importa il file pdf e si estrae il testo da ogni pagina.

pdf_path = "Area_Pre_Clinica_2016.pdf"
reader = PdfReader(pdf_path)

full_text = ""
for page in reader.pages:
    full_text += page.extract_text() + "\n"


In [ ]:
# si utilizza una regex per estrarre le domande e le risposte dal testo.
question_pattern = re.compile(
    r"(?P<id>\d{1,4})\.\s+(?P<question>.+?)\s+A(?P<correct_a>\*)?\s(?P<A>.+?)\s+"
    r"B(?P<correct_b>\*)?\s(?P<B>.+?)\s+"
    r"C(?P<correct_c>\*)?\s(?P<C>.+?)\s+"
    r"D(?P<correct_d>\*)?\s(?P<D>.+?)\s+"
    r"E(?P<correct_e>\*)?\s(?P<E>.+?)(?=\n\d+\.|\Z)",
    re.DOTALL
)


In [ ]:
# Si crea una lista per memorizzare le domande e le risposte estratte.
def macro_category_from_id(qid: int) -> str:
    if 1 <= qid <= 395:
        return "farmacologia"
    elif 396 <= qid <= 1564:
        return "fisiologia"
    elif 1565 <= qid <= 1907:
        return "morfologia"
    elif 1908 <= qid <= 2822:
        return "patologia"
    elif 2823 <= qid <= 3344:
        return "prevenzione e deontologia medica"
    else:
        return ""


In [ ]:
# Si esegue la ricerca delle domande nel testo completo utilizzando la regex definita.
questions = []
for match in question_pattern.finditer(full_text):
    qid = int(match.group("id"))
    correct = ""
    for letter in "abcde":
        if match.group(f"correct_{letter}"):
            correct = letter.upper()
            break

    questions.append({
        "id": qid,
        "question": match.group("question").strip().replace("\n", " "),
        "A": match.group("A").strip().replace("\n", " "),
        "B": match.group("B").strip().replace("\n", " "),
        "C": match.group("C").strip().replace("\n", " "),
        "D": match.group("D").strip().replace("\n", " "),
        "E": match.group("E").strip().replace("\n", " "),
        "answer": correct,
        "category": "Area Preclinica 2016",
        "macro category": macro_category_from_id(qid)
    })


In [ ]:
#  
with open("area_preclinica_2016_domande.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=questions[0].keys())
    writer.writeheader()
    writer.writerows(questions)


In [20]:
df_2016_preclinica = pd.read_csv('area_preclinica_2016_domande.csv')
df_2016_preclinica.head(30)

,id,question,A,B,C,D,E,answer,category,macro category
0,1,In una polmonite da Staphylococcus aureus meti...,vancomicina,oxacillina,gentamicina,piperacillina,claritromicina,A,Area Preclinica 2016,farmacologia
1,2,I farmaci ipoglicemizzanti orali della classe ...,sono sempre impiegati in monoterapia,stimolano la secrezione di insulina e possono ...,sono indicati nel diabete giovanile (tipo I),"a differenza delle biguanidi, non necessitano ...",possono antagonizzare l'effetto ipoglicemizzan...,B,Area Preclinica 2016,farmacologia
2,3,"Quale tra i seguenti farmaci antidiabetici, de...",glipizide,clorpropamide,tolbutamide,metformina,Insulina Lantus,D,Area Preclinica 2016,farmacologia
3,4,Nello shock emorragico il trattamento iniziale...,cristalloidi,colloidi non proteici,albumina,mannitolo,protamina solfato,A,Area Preclinica 2016,farmacologia
4,5,Indicare quale dei seguenti farmaci antitumora...,cisplatino,metotressato,5-fluorouracile,citarabina,6-mercaptopurina,A,Area Preclinica 2016,farmacologia
5,6,Quale tra i seguenti farmaci può causare una g...,clozapina,nizatidina,aloperidolo AREA PRE-CLINICA Pag. 2/565 Minist...,olanzapina,codeina,A,Area Preclinica 2016,farmacologia
6,7,A proposito di un farmaco diuretico (ovvero in...,riduzione della pressione arteriosa.,riduzione dell'edema.,riduzione dell'incidenza di infarti del miocar...,riduzione della mortalità.,aumento dell'escrezione di sodio.,E,Area Preclinica 2016,farmacologia
7,8,Quali dei seguenti farmaci NON possono determi...,betabloccanti,statine,diuretici dell'ansa,benzodiazepine,oppiacei,B,Area Preclinica 2016,farmacologia
8,9,Quale delle seguenti condizioni può essere ind...,ipertensione che non risponde al trattamento c...,vomito da chemioterapici,shock anafilattico,schizofrenia,malattia di Parkinson,E,Area Preclinica 2016,farmacologia
9,10,Quale delle seguenti affermazioni riguardanti ...,Le interazioni tra farmaci non determinano var...,in seguito a un'interazione tra farmaci non pu...,un farmaco non può modificare la biodisponibil...,un farmaco può interferire con l'escrezione di...,un farmaco non può interferire con il metaboli...,D,Area Preclinica 2016,farmacologia


In [ ]:
# puliamo il testo dagli errori
#df_2016_preclinica.info()
df = df_2016_preclinica
regex_da_rimuovere = r"AREA PRE-CLINICA Pag\.\s*\d+\/\d+\s*Ministero dell Istruzione, dell\s*Università e della Ricerca,\s*,"
for column in df[["question", "A", "B", "C", "D", "E", "answer"]]:
    # Controlla se la colonna è di tipo stringa (object in Pandas)
        df[column]=df[column].strip()
       # df[column] = df[column].str.replace(r'\s+', ' ', regex=True)  # Rimuove spazi multipli
        df[column] = df[column].str.replace(regex_da_rimuovere, '', regex=True, flags=re.IGNORECASE)
        # Ho aggiunto flags=re.IGNORECASE per rendere la ricerca non sensibile alle maiuscole/minuscole.
        # Questo è utile se "AREA PRE-CLINICA" a volte è scritto "Area Pre-Clinica", ecc. regex=True)



AttributeError: 'Series' object has no attribute 'strip'

In [88]:
df.to_csv("area_preclinica_2016_domande_pulite.csv", encoding='utf-8')

In [22]:
#Si fa un check per verificare che tutte le domande siano state registrate nel File

inconsistenti = df_2016_preclinica[df_2016_preclinica.index != df_2016_preclinica["id"] - 1]

# Mostra o salva le righe inconsistenti
print(inconsistenti)
#non ci sono osservazioni inconsistenti

Empty DataFrame
Columns: [id, question, A, B, C, D, E, answer, category, macro category]
Index: []


Si fa un check delle domande e delle risposte tra il full text e il il csv ottenuto

In [94]:
# Confrontiamo automaticamente le prime 20 domande tra PDF (testo estratto) e CSV

# Estrarre le domande dal CSV
df_sample = df_2016_preclinica.copy()

# Estrarre lo stesso testo dal PDF già precedentemente letto
# Spezziamo le domande dal testo intero PDF (già caricato in `full_text`)
pdf_questions_raw = question_pattern.findall(full_text)

# Mappiamo l'output del PDF in un dizionario per confronto
pdf_data_map = {}
for q in pdf_questions_raw:
    qid = int(q[0])
    pdf_data_map[qid] = {
        "question": q[1].strip().replace("\n", " "),
        "A": q[3].strip().replace("\n", " "),
        "B": q[5].strip().replace("\n", " "),
        "C": q[7].strip().replace("\n", " "),
        "D": q[9].strip().replace("\n", " "),
        "E": q[11].strip().replace("\n", " "),
    }

# Confronta le prime 20 domande
differenze = []
for _, row in df_sample.iterrows():
    qid = row["id"]
    pdf_entry = pdf_data_map.get(qid)
    if not pdf_entry:
        continue

    diffs = {}
    for key in ["question", "A", "B", "C", "D", "E",]:
        csv_text = str(row[key]).strip()
        pdf_text = str(pdf_entry[key]).strip()
        if csv_text != pdf_text:
            diffs[key] = {"csv": csv_text, "pdf": pdf_text}
    
    if diffs:
        differenze.append({"id": qid, "diffs": diffs})

# Mostriamo le differenze trovate
differenze


[{'id': 6,
  'diffs': {'C': {'csv': 'aloperidolo',
    'pdf': 'aloperidolo AREA PRE-CLINICA Pag. 2/561 Ministero dell Istruzione, dell  Università e della Ricerca, ,'}}},
 {'id': 12,
  'diffs': {'B': {'csv': 'flucloxacillina',
    'pdf': 'flucloxacillina AREA PRE-CLINICA Pag. 3/561 Ministero dell Istruzione, dell  Università e della Ricerca, ,'}}},
 {'id': 18,
  'diffs': {'B': {'csv': "E' sempre irreversibile",
    'pdf': "E' sempre irreversibile AREA PRE-CLINICA Pag. 4/561 Ministero dell Istruzione, dell  Università e della Ricerca, ,"}}},
 {'id': 24,
  'diffs': {'B': {'csv': 'aumento della sintesi proteica a livello cutaneo',
    'pdf': 'aumento della sintesi proteica a livello cutaneo AREA PRE-CLINICA Pag. 5/561 Ministero dell Istruzione, dell  Università e della Ricerca, ,'}}},
 {'id': 30,
  'diffs': {'question': {'csv': 'Cosa si intende per farmacovigilanza?',
    'pdf': 'Cosa si intende per farmacovigilanza? AREA PRE-CLINICA Pag. 6/561 Ministero dell Istruzione, dell  Università 

Si verifica che le categorie siano corrette e le risposte

## Area preclinca 2017 

In [95]:
# Per prima cosa si importa il file pdf e si estrae il testo da ogni pagina.

pdf_path = "Area_Pre_Clinica_2017.pdf"
reader = PdfReader(pdf_path)

full_text = ""
for page in reader.pages:
    full_text += page.extract_text() + "\n"

In [30]:
# si utilizza una regex per estrarre le domande e le risposte dal testo.
question_pattern = re.compile(
    r"(?P<id>\d{1,4})\.\s+(?P<question>.+?)\s+A(?P<correct_a>\*)?\s(?P<A>.+?)\s+"
    r"B(?P<correct_b>\*)?\s(?P<B>.+?)\s+"
    r"C(?P<correct_c>\*)?\s(?P<C>.+?)\s+"
    r"D(?P<correct_d>\*)?\s(?P<D>.+?)\s+"
    r"E(?P<correct_e>\*)?\s(?P<E>.+?)(?=\n\d+\.|\Z)",
    re.DOTALL
)


In [97]:
# Si crea una lista per memorizzare le domande e le risposte estratte.
def macro_category_from_id(qid: int) -> str:
    if 1 <= qid <= 393:
        return "farmacologia"
    elif 394 <= qid <= 1555:
        return "fisiologia"
    elif 1556 <= qid <= 1897:
        return "morfologia"
    elif 1898 <= qid <= 2805:
        return "patologia"
    elif 2806 <= qid <= 3324:
        return "prevenzione e deontologia medica"
    else:
        return ""


In [99]:
# Si esegue la ricerca delle domande nel testo completo utilizzando la regex definita.
questions = []
for match in question_pattern.finditer(full_text):
    qid = int(match.group("id"))
    correct = ""
    for letter in "abcde":
        if match.group(f"correct_{letter}"):
            correct = letter.upper()
            break

    questions.append({
        "id": qid,
        "question": match.group("question").strip().replace("\n", " "),
        "A": match.group("A").strip().replace("\n", " "),
        "B": match.group("B").strip().replace("\n", " "),
        "C": match.group("C").strip().replace("\n", " "),
        "D": match.group("D").strip().replace("\n", " "),
        "E": match.group("E").strip().replace("\n", " "),
        "answer": correct,
        "category": "Area Preclinica 2017",
        "macro category": macro_category_from_id(qid)
    })


In [100]:
#  
with open("area_preclinica_2017_domande.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=questions[0].keys())
    writer.writeheader()
    writer.writerows(questions)


In [101]:
df_2017_preclinica = pd.read_csv('area_preclinica_2017_domande.csv')
df_2017_preclinica.head(30)
#len(df_2017_preclinica)

,id,question,A,B,C,D,E,answer,category,macro category
0,1,In una polmonite da Staphylococcus aureus meti...,vancomicina,oxacillina,gentamicina,piperacillina,claritromicina,A,Area Preclinica 2017,farmacologia
1,2,I farmaci ipoglicemizzanti orali della classe ...,sono sempre impiegati in monoterapia,stimolano la secrezione di insulina e possono ...,sono indicati nel diabete giovanile (tipo I),"a differenza delle biguanidi, non necessitano ...",possono antagonizzare l'effetto ipoglicemizzan...,B,Area Preclinica 2017,farmacologia
2,3,"Quale tra i seguenti farmaci antidiabetici, de...",glipizide,clorpropamide,tolbutamide,metformina,Insulina Lantus,D,Area Preclinica 2017,farmacologia
3,4,Nello shock emorragico il trattamento iniziale...,cristalloidi,colloidi non proteici,albumina,mannitolo,protamina solfato,A,Area Preclinica 2017,farmacologia
4,5,Indicare quale dei seguenti farmaci antitumora...,cisplatino,metotressato,5-fluorouracile,citarabina,6-mercaptopurina,A,Area Preclinica 2017,farmacologia
5,6,Quale tra i seguenti farmaci può causare una g...,clozapina,nizatidina,aloperidolo AREA PRE-CLINICA Pag. 2/561 Minist...,olanzapina,codeina,A,Area Preclinica 2017,farmacologia
6,7,A proposito di un farmaco diuretico (ovvero in...,riduzione della pressione arteriosa.,riduzione dell'edema.,riduzione dell'incidenza di infarti del miocar...,riduzione della mortalità.,aumento dell'escrezione di sodio.,E,Area Preclinica 2017,farmacologia
7,8,Quali dei seguenti farmaci NON possono determi...,betabloccanti,statine,diuretici dell'ansa,benzodiazepine,oppiacei,B,Area Preclinica 2017,farmacologia
8,9,Quale delle seguenti condizioni può essere ind...,ipertensione che non risponde al trattamento c...,vomito da chemioterapici,shock anafilattico,schizofrenia,malattia di Parkinson,E,Area Preclinica 2017,farmacologia
9,10,Quale delle seguenti affermazioni riguardanti ...,Le interazioni tra farmaci non determinano var...,in seguito a un'interazione tra farmaci non pu...,un farmaco non può modificare la biodisponibil...,un farmaco può interferire con l'escrezione di...,un farmaco non può interferire con il metaboli...,D,Area Preclinica 2017,farmacologia


In [ ]:
# puliamo il testo dagli errori
#df_2016_preclinica.info()
df = df_2017_preclinica
regex_da_rimuovere = r"AREA PRE-CLINICA Pag\.\s*\d+\/\d+\s*Ministero dell Istruzione, dell\s*Università e della Ricerca,\s*,"

for column in df[["question", "A", "B", "C", "D", "E", "answer"]]:
    # Controlla se la colonna è di tipo stringa (object in Pandas)
        df[column] = df[column].str.replace(regex_da_rimuovere, '', regex=True, flags=re.IGNORECASE)
        # Ho aggiunto flags=re.IGNORECASE per rendere la ricerca non sensibile alle maiuscole/minuscole.
        # Questo è utile se "AREA PRE-CLINICA" a volte è scritto "Area Pre-Clinica", ecc. regex=True)


In [103]:
inconsistenti = df_2017_preclinica[df_2017_preclinica.index != df_2017_preclinica["id"] - 1]

# Mostra o salva le righe inconsistenti
print(inconsistenti)

Empty DataFrame
Columns: [id, question, A, B, C, D, E, answer, category, macro category]
Index: []


In [104]:
# Confrontiamo automaticamente le prime 20 domande tra PDF (testo estratto) e CSV

# Estrarre le domande dal CSV
df_sample = df_2017_preclinica.copy()

# Estrarre lo stesso testo dal PDF già precedentemente letto
# Spezziamo le domande dal testo intero PDF (già caricato in `full_text`)
pdf_questions_raw = question_pattern.findall(full_text)

# Mappiamo l'output del PDF in un dizionario per confronto
pdf_data_map = {}
for q in pdf_questions_raw:
    qid = int(q[0])
    pdf_data_map[qid] = {
        "question": q[1].strip().replace("\n", " "),
        "A": q[3].strip().replace("\n", " "),
        "B": q[5].strip().replace("\n", " "),
        "C": q[7].strip().replace("\n", " "),
        "D": q[9].strip().replace("\n", " "),
        "E": q[11].strip().replace("\n", " "),
    }

# Confronta le prime 20 domande
differenze = []
for _, row in df_sample.iterrows():
    qid = row["id"]
    pdf_entry = pdf_data_map.get(qid)
    if not pdf_entry:
        continue

    diffs = {}
    for key in ["question", "A", "B", "C", "D", "E",]:
        csv_text = str(row[key]).strip()
        pdf_text = str(pdf_entry[key]).strip()
        if csv_text != pdf_text:
            diffs[key] = {"csv": csv_text, "pdf": pdf_text}
    
    if diffs:
        differenze.append({"id": qid, "diffs": diffs})

# Mostriamo le differenze trovate
differenze


[{'id': 6,
  'diffs': {'C': {'csv': 'aloperidolo',
    'pdf': 'aloperidolo AREA PRE-CLINICA Pag. 2/561 Ministero dell Istruzione, dell  Università e della Ricerca, ,'}}},
 {'id': 12,
  'diffs': {'B': {'csv': 'flucloxacillina',
    'pdf': 'flucloxacillina AREA PRE-CLINICA Pag. 3/561 Ministero dell Istruzione, dell  Università e della Ricerca, ,'}}},
 {'id': 18,
  'diffs': {'B': {'csv': "E' sempre irreversibile",
    'pdf': "E' sempre irreversibile AREA PRE-CLINICA Pag. 4/561 Ministero dell Istruzione, dell  Università e della Ricerca, ,"}}},
 {'id': 24,
  'diffs': {'B': {'csv': 'aumento della sintesi proteica a livello cutaneo',
    'pdf': 'aumento della sintesi proteica a livello cutaneo AREA PRE-CLINICA Pag. 5/561 Ministero dell Istruzione, dell  Università e della Ricerca, ,'}}},
 {'id': 30,
  'diffs': {'question': {'csv': 'Cosa si intende per farmacovigilanza?',
    'pdf': 'Cosa si intende per farmacovigilanza? AREA PRE-CLINICA Pag. 6/561 Ministero dell Istruzione, dell  Università 

### Proviamo a mergiare i due dataset per vedere quali domande sono in comune e quali no

In [105]:
len(df_2016_preclinica)
len(df_2017_preclinica)

3324

In [108]:


# Unisci i due DataFrame
df_merged = pd.concat([df_2016_preclinica,  df_2017_preclinica])
len(df_merged)

mask_diff = ~df_merged.duplicated(subset=["question", "answer"], keep=False)
df_differenze = df_merged[mask_diff]
df_differenze #

filter=df_differenze["category"]== "Area Preclinica 2017"
df_differenze = df_differenze[filter]
df_differenze
df_differenze.to_csv("differenze_area_preclinica_2016_2017.csv", index=False, encoding='utf-8')


#Rimuovi i duplicati (tutte le colonne devono essere uguali)

#df_merged = df_merged.drop_duplicates()
#len(df_merged)


3330,Non è un inquinante atmosferico irritante per le vie respiratorie: ,O3,NO2,SO2,CO,H2SO4,D,Area Preclinica 2016,prevenzione e deontologia medica
3310,Non è un inquinante atmosferico irritante per le vie respiratorie:,O3,NO2,SO2,CO,H2SO4,D,Area Preclinica 2017,prevenzione e deontologia medica
3314,L'uso più idoneo del latte di calce consiste: ,Nella disinfezione dell'acqua,Nella disinfezione di materiale fecale e pozzi neri,Nella disinfezione di ambienti ospedalieri,Nella disinfezione di ferri chirurgici,Nella disinfestazione di ambienti,B,Area Preclinica 2017,prevenzione e deontologia medica